# Data Preprocessing

The goal of this lab is to introduce you to data preprocessing techniques in order to make your data suitable for applying a learning algorithm.

## 1. Handling Missing Values

A common (and very unfortunate) data property is the ocurrence of missing and erroneous values in multiple features in our dataset.
Download the dataset and corresponding information from the <a href="http://www.cs.uni-potsdam.de/ml/teaching/ss15/ida/uebung02/abalone.csv">course website</a>.

To determine the age of a abalone snail you have to kill the snail and count the annual
rings. You are told to estimate the age of a snail on the basis of the following attributes:
1. type: male (0), female (1) and infant (2)
2. length in mm
3. width in mm
4. height in mm
5. total weight in grams
6. weight of the meat in grams
7. drained weight in grams
8. weight of the shell in grams
9. number of annual rings (number of rings +1, 5 yields age)

However, these data is incomplete. Missing values are marked with −1.

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt

# load data 
df = pd.read_csv("abalone.csv")
df.columns=['type','length','width','height','total_weight','meat_weight','drained_weight','shell_weight','num_rings']
df.head()

,type,length,width,height,total_weight,meat_weight,drained_weight,shell_weight,num_rings
0,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,-1
1,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,2,-1.000,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,2,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


### Exercise 1.1

Compute the mean of all positive numbers of each numeric column and the counts of each category.

In [267]:
print("Mean of all positive numbers of each numeric column\n")
print(df[df.iloc[:,1:8] > 0].mean()[1:8],"\n")
print("The counts of each category\n")
print(df.groupby(['type'])[['type']].count())

Mean of all positive numbers of each numeric column

length            0.523692
width             0.407955
height            0.139679
total_weight      0.828843
meat_weight       0.359263
drained_weight    0.180249
shell_weight      0.238604
dtype: float64 

The counts of each category

      type
type      
-1      87
 0    1500
 1    1279
 2    1310


### Exercise 1.2

Compute the median of all positive numbers of each numeric column.

In [268]:
print("Median of all positive numbers of each numeric column\n")
print(df[df.iloc[:,1:8] > 0].median()[1:8],"\n")

Median of all positive numbers of each numeric column

length            0.54500
width             0.42500
height            0.14000
total_weight      0.80175
meat_weight       0.33600
drained_weight    0.17050
shell_weight      0.23350
dtype: float64 



### Exercise 1.3

Handle the missing values in a way that you find suitable. Argue your choices.

In [25]:
#transorm to numpy array
#data
X = df.loc[df["type"] >= 0,"length":].values
#target
y = df.loc[df["type"] >= 0,"type"].values
X_missing = df.loc[df["type"] == -1,"length":].values

In [3]:
print("about",np.round(X_missing.shape[0] / df.shape[0] * 100, decimals = 2),"% the data is missing")

about 2.08 % the data is missing


I dropped it because first it only missing 2% and other classifier cant predict significantlty the missing gender

In [4]:
df.loc[df["type"] < 0,"type"] = np.nan

In [5]:
df = df.dropna()

In [26]:
#prepare the data for modeling
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1, random_state=42, stratify=y )

In [30]:
#KneighborClassifiert
knn = KNeighborsClassifier(n_neighbors=14)
knn.fit(X_train,y_train)
#print(knn.score(X_test, y_test))
y_predict = knn.predict(X_test)
print("Confusion Matrix")
print(classification_report(y_test,y_predict))

Confusion Matrix
              precision    recall  f1-score   support

         0.0       0.47      0.57      0.51       150
         1.0       0.40      0.30      0.34       128
         2.0       0.73      0.73      0.73       131

   micro avg       0.54      0.54      0.54       409
   macro avg       0.53      0.53      0.53       409
weighted avg       0.53      0.54      0.53       409



In [8]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_predict = logreg.predict(X_test)
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.44      0.65      0.52       150
           1       0.11      0.02      0.03       128
           2       0.60      0.78      0.68       131

   micro avg       0.49      0.49      0.49       409
   macro avg       0.38      0.48      0.41       409
weighted avg       0.39      0.49      0.42       409



/home/pandoora/anaconda3/envs/mi/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pandoora/anaconda3/envs/mi/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [9]:
#if those ware predictable:
#y_filling = model.predict(X_missing)
#cleaned_df = pd.DataFrame(data= np.c_[y_filling, X_missing],columns = df.columns)

### Exercise 1.4

Perform Z-score normalization on every column (except the type of course!)

In [19]:
means = list(df.loc[:,"length":"num_rings"].mean())
std   = list(df.loc[:,"length":"num_rings"].std())

print("Mean ",df.loc[:,"length":"num_rings"].mean())
print("Standard Dev",df.loc[:,"length":"num_rings"].std())
cols = list(df.columns)
cols.remove('type')

Mean  length            0.478210
width             0.366149
height            0.105713
total_weight      0.783750
meat_weight       0.319148
drained_weight    0.149871
shell_weight      0.209308
num_rings         9.670580
dtype: float64
Standard Dev length            0.286111
width             0.258855
height            0.198261
total_weight      0.563480
meat_weight       0.319780
drained_weight    0.216505
shell_weight      0.234247
num_rings         3.603491
dtype: float64


In [254]:
for col in cols:
    col_zscore = col + '_zscore'
    #forula for Z Score
    df[col_zscore] = (df[col] - df[col].mean())/df[col].std()
    
print(df.head())


    02  041300   07  0815   10  101  10511        11  115397   12  ...  \
0  0.0     0.0  0.0   0.0  0.0  0.0    0.0  0.000000     0.0  0.0  ...   
1  0.0     0.0  0.0   0.0  0.0  0.0    0.0  0.000000     0.0  0.0  ...   
2  0.0     0.0  0.0   0.0  0.0  0.0    0.0  3.302585     0.0  0.0  ...   
3  0.0     0.0  0.0   0.0  0.0  0.0    0.0  0.000000     0.0  0.0  ...   
4  0.0     0.0  0.0   0.0  0.0  0.0    0.0  0.000000     0.0  0.0  ...   

   yellow_zscore  yeltsin_zscore  yet_zscore  you_zscore  young_zscore  \
0      -0.223607       -0.223607   -0.223607   -0.670332     -0.324893   
1      -0.223607       -0.223607   -0.223607   -0.670332     -0.324893   
2      -0.223607       -0.223607   -0.223607   -0.670332      2.924038   
3      -0.223607       -0.223607   -0.223607    3.272799     -0.324893   
4      -0.223607       -0.223607   -0.223607   -0.670332     -0.324893   

   younger_zscore  your_zscore  z3_zscore  zeta_zscore  zeus_zscore  
0       -0.223607    -0.223607  -0.22360

## 2. Preprocessing text (Optional)

One possible way to transform text documents into vectors of numeric attributes is to use the TF-IDF representation. We will experiment with this representation using the 20 Newsgroup data set. The data set contains postings on 20 different topics. The classification problem is to decide which of the topics a posting falls into. Here, we will only consider postings about medicine and space.

In [212]:
from sklearn.datasets import fetch_20newsgroups
import math

categories = ['sci.med', 'sci.space']
raw_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
print('The index of each category is: {}'.format([(i,target) for i,target in enumerate(raw_data.target_names)]))

The index of each category is: [(0, 'sci.med'), (1, 'sci.space')]


Check out some of the postings, might find some funny ones!

In [40]:
import numpy as np
idx = np.random.randint(0, len(raw_data.data))
print ('This is a {} email.\n'.format(raw_data.target_names[raw_data.target[idx]]))
print ('There are {} emails.\n'.format(len(raw_data.data)))
print(raw_data.data[idx])

This is a sci.med email.

There are 1187 emails.

From: smithmc@mentor.cc.purdue.edu (Lost Boy)
Subject: Re: Can men get yeast infections?
Organization: Purdue University Computing Center
Distribution: na
Lines: 25

In article <noringC5Fnx2.2v2@netcom.com> noring@netcom.com (Jon Noring) writes:
>In article Tammy.Vandenboom@launchpad.unc.edu (Tammy Vandenboom) writes:
>
>>Here's a potentially stupid question to possibly the wrong news group, but. .
>>
>>Can men get yeast infections? Spread them? What kind of symptoms?
>>Similar as women's?  I have a yeast infection and my husband (who is a
>>natural paranoid on a good day) is sure he's gonna catch it and keeps
>>asking me what it's like.  I'm not sure what his symptoms would be. . 
>
>The answer is yes and no.  I'm sure others on sci.med can expand on this.
>
>Jon

I know from personal experience that men CAN get yeast infections. I 
get rather nasty ones from time to time, mostly in the area of the
scrotum and the base of the penis. Th

Lets pick the first 10 postings from each category

In [57]:
idxs_med = np.flatnonzero(raw_data.target == 0)
idxs_space = np.flatnonzero(raw_data.target == 1)
idxs = np.concatenate([idxs_med[:10],idxs_space[:10]])
data = np.array(raw_data.data)
data = data[idxs]

<a href="http://www.nltk.org/">NLTK</a> is a toolkit for natural language processing. Take some time to install it and go through this <a href="http://www.slideshare.net/japerk/nltk-in-20-minutes">short tutorial/presentation</a>.

The downloaded package below is a tokenizer that divides a text into a list of sentences, by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences.

In [222]:
import nltk
import itertools
nltk.download('punkt')

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in data]
vocabulary_size = 1000
unknown_token = 'unknown'

[nltk_data] Downloading package punkt to /home/pandoora/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [223]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print ("Found %d unique words tokens." % len(word_freq.items()))

Found 1641 unique words tokens.


In [224]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
 
print ("Using vocabulary size %d." % vocabulary_size)
print ("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

Using vocabulary size 1000.
The least frequent word in our vocabulary is 'REASONS' and appeared 1 times.


In [225]:
import collections
od = collections.OrderedDict(sorted(word_to_index.items()))

### Exercise 2.1

Code your own TF-IDF representation function and use it on this dataset. (Don't use code from libraries. Build your own function with Numpy/Pandas). Use the formular TFIDF = TF * (IDF+1). The effect of adding “1” to the idf in the equation above is that terms with zero idf, i.e., terms that occur in all documents in a training set, will not be entirely ignored.

In [258]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()
df = pd.DataFrame(countvec.fit_transform(data).toarray(), columns=countvec.get_feature_names())

In [259]:
def tfidf(df):
    for col in list(df.columns):
        #forula for for idf
        df[col] = df[col] * (math.log(20/sum(df[col] != 0)) +1)    
    return df

rep = tfidf(df)

# Check if your implementation is correct
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm=None, smooth_idf=False, use_idf=True)
X_train = pd.DataFrame(vectorizer.fit_transform(data).toarray(), columns=countvec.get_feature_names())
answer=['No','Yes']
if rep is not None:
    print ('Is this implementation correct?\nAnswer: {}'.format(answer[1*np.all(X_train == rep)]))

Is this implementation correct?
Answer: Yes
